In [7]:
from dataclasses import dataclass
from typing import List, Dict, Tuple

In [8]:
@dataclass(order=True)
class Process:
  pid: str
  arrival: int
  cpu_burst: int

In [9]:
def sjf(processes):
  # Copy and sort process by (arrival, pid)
  not_arrived_processes = sorted(processes, key=lambda p: (p.arrival, p.pid))

  # processes currently eligible to run (arrival <= time)
  ready_processes = []

  # The resulting process schedule
  scheduled_processes = []

  # completion time of processes
  completion = {}

  # current time
  t = 0

  # Loop until there are no processes left to run
  while not_arrived_processes or ready_processes:
    # Admit all processes whose arrival time has passes (arrival <= t)
    while not_arrived_processes and not_arrived_processes[0].arrival <= t:
      # Move the process into the ready list
      ready_processes.append(not_arrived_processes.pop(0))

    # If CPU would be idle (no ready processes)
    if not ready_processes:
      t = not_arrived_processes[0].arrival # Jump time to the next process's time
      continue # re-check arrivals

    # Choose the next process
    ready_processes.sort(key=lambda p: (p.cpu_burst, p.arrival, p.pid))
    # Take the process with the shorted cpu burst
    the_ready_process = ready_processes.pop(0)

    # Simulate
    start = t
    end = start + the_ready_process.cpu_burst
    scheduled_processes.append((the_ready_process.pid, start, end))
    completion[the_ready_process.pid] = end

    t = end # Advance the time to the end of the process

  return scheduled_processes, completion

In [10]:
def tat(processes, completion):
  return {p.pid: completion[p.pid] - p.arrival for p in processes} # Map each PID to its TAT.

def att(tats):
    # Average TAT is the mean over all processes (0.0 if there are none).
    return (sum(tats.values()) / len(tats)) if tats else 0.0

In [11]:
example = [
    Process('p1', arrival=0, cpu_burst=3),
    Process('p2', arrival=2, cpu_burst=6),
    Process('p3', arrival=4, cpu_burst=4),
    Process('p4', arrival=6, cpu_burst=5),
    Process('p5', arrival=8, cpu_burst=2),
]
scheduled_processes, completion = sjf(example)
tats = tat(example, completion)
att = att(tats)

In [12]:
print('TAT: ', tats)
print('ATT: ', att)

TAT:  {'p1': 3, 'p2': 7, 'p3': 11, 'p4': 14, 'p5': 3}
ATT:  7.6
